# Analysis of the Regulatory Market model

> Initial results for my Regulatory Markets model

In [1]:
#| default_exp analysis_stochasticgames

In [2]:
#| export
from gh_pages_example.conditions import *
from gh_pages_example.data_utils import *
from gh_pages_example.methods import *
from gh_pages_example.models import *
from gh_pages_example.model_utils import *
from gh_pages_example.payoffs import *
from gh_pages_example.plot_utils import *
from gh_pages_example.types import *
from gh_pages_example.utils import *

import copy
import typing
import warnings

import chaospy
import fastcore.test
import ipywidgets
from ipywidgets import interact
import matplotlib as mpl
import matplotlib.pyplot as plt
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas
import scipy
import seaborn as sns

/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/model_utils.py:299: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


In [3]:
#| hide
sns.set_theme(style='darkgrid',palette='deep' ,font='sans-serif', font_scale=1.4)
plt.rcParams["axes.grid"] = False

## Fast prototyping (before placing in main body of notebook)

#### Flow Payoffs

In [4]:
#| export payoffs
@method(build_payoffs, 'vasconcelos_2014_flow')
def build_payoffs(models: dict):
    names = ['payoffs_state', 'c', 'T', 'b_r', 'b_p', 'r', 'g']
    payoffs_state, c, T, b_r, b_p, r, g = [models[k] for k in names]
    strategy_counts = payoffs_state['strategy_counts']
    state = payoffs_state['state']
    reward_bonus = g if state=='1' else 1
    n_r = strategy_counts.get("2", 0)
    n_p = strategy_counts.get("4", 0)
    risk = r * (n_r * c * b_r + n_p * c * b_p < T)
    payoffs = {"1": (1 - risk) * b_r * reward_bonus,  # rich_free_rider
               "2": (1 - risk) * c * b_r * reward_bonus,  # rich_contributor
               "3": (1 - risk) * b_p * reward_bonus,  # poor_free_rider
               "4": (1 - risk) * c * b_p * reward_bonus}  # poor_contributor
    return {**models, "flow_payoffs": payoffs}

#### Dynamics

In [5]:
@method(state_transition, 'ex1')
def state_transition(models):
    """Compute transition likelihood for a model with 2 states and an arbitrary
    number of players. To stay in the good state, 0, all players need to choose
    to cooperate, i.e. action 1."""
    state_action, next_state = [models[k] for k in ['state_action',
                                                    'next_state']]
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and next_state == '1'
        and n_cooperators != n_players):
        transition_likelihood = 1
    elif (current_state == '1'
          and next_state == '0'
          and n_cooperators == n_players):
        transition_likelihood = 1
    elif (current_state == '0'
          and next_state == '0'
          and n_cooperators == n_players):
        transition_likelihood = 1
    elif (current_state == '1'
          and next_state == '1'
          and n_cooperators != n_players):
        transition_likelihood = 1
    else:
        transition_likelihood = 0
    return transition_likelihood

#### Strategies

In [6]:
@method(build_strategy, 'ex1_rich_cooperator')
def build_strategy(models):
    """A rich player who cooperates with 95% probability if everyone currently
    cooperates, otherwise defects with 95% probability."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A1": 0.95, "A2": 0.05}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A1": 0.95, "A2": 0.05}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    return strategy

@method(build_strategy, 'ex1_rich_defector')
def build_strategy(models):
    """A rich player who defects with 95% probability no matter what others
    do, nor what state they are in."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A1": 0.05, "A2": 0.95}
    return strategy

@method(build_strategy, 'ex1_poor_cooperator')
def build_strategy(models):
    """A poor player who cooperates with 95% probability if everyone currently
    cooperates, otherwise defects with 95% probability."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A3": 0.95, "A4": 0.05}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A3": 0.95, "A4": 0.05}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    return strategy

@method(build_strategy, 'ex1_poor_defector')
def build_strategy(models):
    """A poor player who defects with 95% probability no matter what others
    do, nor what state they are in."""
    state_action = models['state_action']
    current_state, action_profile = state_action.split(":")
    action_tuple = string_to_tuple(action_profile)
    action_count = dict(zip(*np.unique(action_tuple, return_counts=True)))
    n_players = len(action_tuple)
    n_cooperators = action_count.get(1, 0) + action_count.get(3, 0)
    if (current_state == '0'
        and n_cooperators == n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '0'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators == n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    elif (current_state == '1'
          and n_cooperators != n_players):
        strategy = {"A3": 0.05, "A4": 0.95}
    return strategy

#### Model in full

In [7]:
Z = {"S2": 50, "S1": 50}
β = 1
# In this case the sector strategies for the full stochsatic game happen to
# coincide with the actions allowed each period.
sector_strategies = {"S1": ["1", "2"],
                     "S2": ["3", "4"]}
allowed_sectors = {"P1": ["S1", "S2"],
                   "P2": ["S1", "S2"]}
models = {"allowed_sectors": allowed_sectors,
          "sector_strategies": sector_strategies,
          "profiles_rule": "anonymous", }
action_profiles = create_profiles(models)["profiles"]
n_states = 2
state_actions = [f"{state}:{a}"
                 for a in action_profiles
                 for state in range(n_states)]

strategy_keys = ["ex1_rich_cooperator",
                 "ex1_rich_defector",
                 "ex1_poor_cooperator",
                 "ex1_poor_defector", ]
models = {**models,
          "strategy_keys": strategy_keys,
          "state_actions": state_actions}
strategies = build_strategies(models)['strategies']
models = {**models,
          "payoffs_flow_key": "vasconcelos_2014_flow",
          "payoffs_key": "flow_payoffs_wrapper",
          "state_actions": state_actions,
          "strategies": strategies,
          'n_states': n_states,
          'state_transition_key': 'ex1',
          'compute_transition_key': "anonymous_actions",
          **model_builder({'c': np.arange(0,1,0.02),
                           'T': 1,
                           'b_r': 4,
                           'b_p': 2,
                           'r': np.arange(0.5, 1, 0.02),
                           'g': np.arange(1, 2, 0.2),
                           "β": β,})
          }
models
models = build_state_transitions(models)
models = build_payoffs(models)
models = {**models,
          "payoffs_key": "payoff_function_wrapper",
          "profile_payoffs_key": "stochastic-no-discounting",
          "dispatch-type": 'multiple-populations',
          "Z": Z, }
models = build_payoffs(models)

/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:260: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


In [8]:
results = thread_macro(models,
                       create_profiles,
                       apply_profile_filters,
                       build_transition_matrix,
                       calculate_sd_helper,
                       )

df = thread_macro(results,
                  results_to_dataframe_egt,)

/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/model_utils.py:299: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:311: RuntimeWarning: Some of the entries in the transition matrix are close to 1 (with a tolerance of 1e-11). This could result in more than one eigenvalue of magnitute 1 (the Markov Chain is degenerate), so please be careful when analysing the results.
  warn(


In [9]:
df

,c,r,g,T,b_r,b_p,β,3-1_frequency,3-2_frequency,4-1_frequency,4-2_frequency
0,0.00,0.50,1.0,1,4,2,1,1.000000,1.273387e-27,5.675685e-14,2.548105e-40
1,0.00,0.50,1.2,1,4,2,1,1.000000,0.000000e+00,0.000000e+00,0.000000e+00
2,0.00,0.50,1.4,1,4,2,1,1.000000,0.000000e+00,0.000000e+00,0.000000e+00
3,0.00,0.50,1.6,1,4,2,1,1.000000,0.000000e+00,0.000000e+00,0.000000e+00
4,0.00,0.50,1.8,1,4,2,1,1.000000,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
6245,0.98,0.98,1.0,1,4,2,1,0.259223,2.468767e-01,2.529745e-01,2.409258e-01
6246,0.98,0.98,1.2,1,4,2,1,0.259626,2.468674e-01,2.533661e-01,2.401401e-01
6247,0.98,0.98,1.4,1,4,2,1,0.260031,2.468563e-01,2.537558e-01,2.393572e-01
6248,0.98,0.98,1.6,1,4,2,1,0.260436,2.468436e-01,2.541437e-01,2.385769e-01


In [10]:
var = "g"

In [11]:
@interact
def plot_heatmap_data(v=df[var].unique()):
    plot_data = df[df[var]==v]
    table = plot_data.pivot_table(index='r', columns='c', values='3-1_frequency')
    plot_heatmap(table,
                    xlabel='Contribution cost',
                    ylabel='Risk',
                    zlabel='Frequency of full cooperation',
                    cmap='inferno',
                    interpolation="nearest"
                    )

interactive(children=(Dropdown(description='v', options=(1.0, 1.2, 1.4, 1.5999999999999999, 1.7999999999999998…

#| hide

# References

In [13]:
#| hide
# import nbdev; nbdev.nbdev_export()